# QCEW Data Importer

## Introduction

This script is meant to take Quarterly Census of Employment and Wages data to filter for 15-County region

### Process Outline

The process carried out by this workflow can be described as follows:
  - The script will retrieve the QCEW datasets from the US Bureau of Labor Statistics
  - For each County and year specified, a ".csv" is created with a corresponding ".yaml" schema and  ".yaml" resource file

## Setup

### Import required packages

In [17]:
import csv
import os
import urllib
import pandas as pd
import yaml
from tableschema import Table
from frictionless import Resource, validate
import sys
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### Parameters

In [27]:
# Define input and output directories
INPUT_DIR = "./input_data"
OUTPUT_DIR = "./output_data"

# Define years for data collection
YEARS_PARAMETER = {
    "2015-2021": [2015, 2016, 2017, 2018, 2019, 2020, 2021],
    "2020":[2020],
}

# Define FIPS codes for data collection
CONST_REGIONS_CODES = {
    "15-County Region": ["39041", "39045", "39047", "39049", "39073", "39083", "39089", "39091", "39097", "39101", "39117", "39127", "39129", "39141", "39159"],
}

# Annual and Quarter codes for looping
ALL_QUARTERS = {
    "QUARTERS": [1, 2, 3, 4],
    "ANNUAL": ["a"],
}

# Schemas for Annual and Quarterly QCEW data
SCHEMA_PARAMETER = {
    "QUATERS": [
    {"name": "area_fips", "type": "string", "description": "5-character FIPS code", "maxLength": 5},
    {"name": "own_code", "type": "string", "description": "1-character ownership code", "maxLength": 1},
    {"name": "industry_code", "type": "string", "description": "6-character industry code (NAICS, SuperSector)", "maxLength": 6},
    {"name": "agglvl_code", "type": "string", "description": "2-character aggregation level code", "maxLength": 2},
    {"name": "size_code", "type": "string", "description": "1-character size code", "maxLength": 1},
    {"name": "year", "type": "string", "description": "4-character year", "maxLength": 4},
    {"name": "qtr", "type": "string", "description": "1-character quarter (always A for annual)", "maxLength": 1},
    {"name": "disclosure_code", "type": "string", "description": "1-character disclosure code (either ' ' (blank) or 'N' not disclosed)", "maxLength": 1},
    {"name": "qtrly_estabs", "type": "number", "description": "Count of establishments for a given quarter"},
    {"name": "month1_emplvl", "type": "number", "description": "Employment level for the first month of a given quarter"},
    {"name": "month2_emplvl", "type": "number", "description": "Employment level for the second month of a given quarter"},
    {"name": "month3_emplvl", "type": "number", "description": "Employment level for the third month of a given quarter"},
    {"name": "total_qtrly_wages", "type": "number", "description": "Total wages for a given quarter"},
    {"name": "taxable_qtrly_wages", "type": "number", "description": "Taxable wages for a given quarter"},
    {"name": "qtrly_contributions", "type": "number", "description": "Quarterly contributions for a given quarter"},
    {"name": "avg_wkly_wage", "type": "number", "description": "Average weekly wage for a given quarter"},
    {"name": "lq_disclosure_code", "type": "string", "description": "1-character location-quotient disclosure code (either ' ' (blank) or 'N' not disclosed)"},
    {"name": "lq_qtrly_estabs", "type": "number", "description": "Location quotient of the quarterly establishment count relative to the U.S."},
    {"name": "oty_disclosure_code", "type": "string", "description": "1-character over-the-year disclosure code (either ' ' (blank) or 'N' not disclosed)"},
    {"name": "oty_qtrly_estabs_chg", "type": "number", "description": "Over-the-year change in the count of establishments for a given quarter"},
    {"name": "oty_qtrly_estabs_pct_chg", "type": "number", "description": "Over-the-year percent change in the count of establishments for a given quarter"},
    {"name": "oty_month1_emplvl_chg", "type": "number", "description": "Over-the-year change in the first month's employment level of a given quarter"},
    {"name": "oty_month1_emplvl_pct_chg", "type": "number", "description": "Over-the-year percent change in the first month's employment level of a given quarter"},
    {"name": "oty_month2_emplvl_chg", "type": "number", "description": "Over-the-year change in the second month's employment level of a given quarter"},
    {"name": "oty_month2_emplvl_pct_chg", "type": "number", "description": "Over-the-year percent change in the second month's employment level of a given quarter (Rounded to the tenths place)"},
    {"name": "oty_month3_emplvl_chg", "type": "number", "description": "Over-the-year change in the third month's employment level of a given quarter"},
    {"name": "oty_month3_emplvl_pct_chg", "type": "number", "description": "Over-the-year percent change in the third month's employment level of a given quarter (Rounded to the tenths place)"},
    {"name": "oty_total_qtrly_wages_chg", "type": "number", "description": "Over-the-year change in total quarterly wages for a given quarter"},
    {"name": "oty_total_qtrly_wages_pct_chg", "type": "number", "description": "Over-the-year percent change in total quarterly wages for a given quarter (Rounded to the tenths place)"},
    {"name": "oty_taxable_qtrly_wages_chg", "type": "number", "description": "Over-the-year change in taxable quarterly wages for a given quarter"},
    {"name": "oty_taxable_qtrly_wages_pct_chg", "type": "number", "description": "Over-the-year percent change in taxable quarterly wages for a given quarter (Rounded to the tenths place)"},
    {"name": "oty_qtrly_contributions_chg", "type": "number", "description": "Over-the-year change in quarterly contributions for a given quarter"},
    {"name": "oty_qtrly_contributions_pct_chg", "type": "number", "description": "Over-the-year percent change in quarterly contributions for a given quarter (Rounded to the tenths place)"},
    {"name": "oty_avg_wkly_wage_chg", "type": "number", "description": "Over-the-year change in average weekly wage for a given quarter"},
    {"name": "oty_avg_wkly_wage_pct_chg", "type": "number", "description": "Over-the-year percent change in average weekly wage for a given quarter (Rounded to the tenths place)"}
],
    "ANNUAL": [
    {"name": "area_fips", "type": "string", "description": "5-character FIPS code", "maxLength": 5},
    {"name": "own_code", "type": "string", "description": "1-character ownership code", "maxLength": 1},
    {"name": "industry_code", "type": "string", "description": "6-character industry code (NAICS, SuperSector)", "maxLength": 6},
    {"name": "agglvl_code", "type": "string", "description": "2-character aggregation level code", "maxLength": 2},
    {"name": "size_code", "type": "string", "description": "1-character size code", "maxLength": 1},
    {"name": "year", "type": "string", "description": "4-character year", "maxLength": 4},
    {"name": "qtr", "type": "string", "description": "1-character quarter (always A for annual)", "maxLength": 1},
    {"name": "disclosure_code", "type": "string", "description": "1-character disclosure code (either ' ' (blank) or 'N' not disclosed)", "maxLength": 1},
    {"name": "annual_avg_estabs", "type": "number", "description": "Annual average of quarterly establishment counts for a given year"},
    {"name": "annual_avg_emplvl", "type": "number", "description": "Annual average of monthly employment levels for a given year"},
    {"name": "total_annual_wages", "type": "number", "description": "Sum of the four quarterly total wage levels for a given year"},
    {"name": "taxable_annual_wages", "type": "number", "description": "Sum of the four quarterly total taxable wage totals for a given year"},
    {"name": "annual_contributions", "type": "number", "description": "Sum of the four quarterly contribution totals for a given year"},
    {"name": "annual_avg_wkly_wage", "type": "number", "description": "Average weekly wage based on the 12-monthly employment levels and total annual wage levels"},
    {"name": "avg_annual_pay", "type": "number", "description": "Average annual pay based on employment and wage levels for a given year"},
    {"name": "lq_disclosure_code", "type": "string", "description": "1-character location-quotient disclosure code (either '' (blank) or 'N' not disclosed)", "maxLength": 1},
    {"name": "lq_annual_avg_estabs", "type": "number", "description": "Location quotient of annual average establishment count relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_annual_avg_emplvl", "type": "number", "description": "Location quotient of annual average employment relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_total_annual_wages", "type": "number", "description": "Location quotient of total annual wages relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_taxable_annual_wages", "type": "number", "description": "Location quotient of taxable annual wages relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_annual_contributions", "type": "number", "description": "Location quotient of total annual contributions relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_annual_avg_wkly_wage", "type": "number", "description": "Location quotient of annual average weekly wage relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "lq_avg_annual_pay", "type": "number", "description": "Location quotient of annual average pay relative to the U.S. (Rounded to the hundredths place)"},
    {"name": "oty_disclosure_code", "type": "string", "description": "1-character over-the-year disclosure code (either ' ' (blank) or 'N' not disclosed)", "maxLength": 1},
    {"name": "oty_annual_avg_estabs_chg", "type": "number", "description": "Over-the-year change in annual average establishments for a given year"},
    {"name": "oty_annual_avg_estabs_pct_chg", "type": "number", "description": "Over-the-year percent change in annual average establishments for a given year (Rounded to the tenths place)"},
    {"name": "oty_annual_avg_emplvl_chg", "type": "number", "description": "Over-the-year change in annual average employment for a given year"},
    {"name": "oty_annual_avg_emplvl_pct_chg", "type": "number", "description": "Over-the-year percent change in annual average employment for a given year (Rounded to the tenths place)"},
    {"name": "oty_total_annual_wages_chg", "type": "number", "description": "Over-the-year change in the total annual wages for a given year"},
    {"name": "oty_total_annual_wages_pct_chg", "type": "number", "description": "Over-the-year percent change in total annual wages for a given year (Rounded to the tenths place)"},
    {"name": "oty_taxable_annual_wages_chg", "type": "number", "description": "Over-the-year change in taxable annual wages for a given year"},
    {"name": "oty_taxable_annual_wages_pct_chg", "type": "number", "description": "Over-the-year percent change in taxable annual wages for a given year (Rounded to the tenths place)"},
    {"name": "oty_annual_contributions_chg", "type": "number", "description": "Over-the-year change in annual contributions for a given year"},
    {"name": "oty_annual_contributions_pct_chg", "type": "number", "description": "Over-the-year percent change in annual contributions for a given year (Rounded to the tenths place)"},
    {"name": "oty_annual_avg_wkly_wage_chg", "type": "number", "description": "Over-the-year change in annual average weekly wage for a given year"},
    {"name": "oty_annual_avg_wkly_wage_pct_chg", "type": "number", "description": "Over-the-year percent change in annual average weekly wage for a given year (Rounded to the tenths place)"},
    {"name": "oty_avg_annual_pay_chg", "type": "number", "description": "Over-the-year change in average annual pay for a given year"},
    {"name": "oty_avg_annual_pay_pct_chg", "type": "number", "description": "Over-the-year percent change in average annual pay for a given year (Rounded to the tenths place)"}
],
}

QCEW_TABLE_DOC_URL="https://www.bls.gov/cew/additional-resources/open-data/csv-data-slices.htm"




### Define Outputs

Data, schema, and resource files for each county, year, and quaters selected

In [19]:
TEMP_TABLE_FILENAME = "morpc-qcew_{year}_{qtr}_{region}.csv"
TEMP_TABLE_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_FILENAME)
TEMP_TABLE_SCHEMA_FILENAME = TEMP_TABLE_FILENAME.replace(".csv",".schema.yaml")
TEMP_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_SCHEMA_FILENAME)
TEMP_TABLE_RESOURCE_FILENAME = TEMP_TABLE_FILENAME.replace(".csv",".resource.yaml")
TEMP_TABLE_RESOURCE_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_RESOURCE_FILENAME)
print("Data: {}".format(TEMP_TABLE_PATH))
print("Schema: {}".format(TEMP_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(TEMP_TABLE_RESOURCE_PATH))

Data: ./output_data\morpc-qcew_{year}_{qtr}_{region}.csv
Schema: ./output_data\morpc-qcew_{year}_{qtr}_{region}.schema.yaml
Resource file: ./output_data\morpc-qcew_{year}_{qtr}_{region}.resource.yaml


### Hashcode Method

In [20]:
import hashlib

def compute_md5(file_path):
    """Compute MD5 hash of the specified file."""
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

## Getting input data

### Calling API and Creating Rows

In [21]:
# *******************************************************************************
# qcewCreateDataRows : This function takes a raw csv string and splits it into
# a two-dimensional array containing the data and the header row of the csv file
# a try/except block is used to handle for both binary and char encoding
def qcewCreateDataRows(csv):
    dataRows = []
    try: dataLines = csv.decode().split('\r\n')
    except er: dataLines = csv.split('\r\n');
    for row in dataLines:
        dataRows.append(row.split(','))
    return dataRows
# *******************************************************************************


# *******************************************************************************
# qcewGetAreaData : This function takes a year, quarter, and area argument and
# returns an array containing the associated area data. Use 'a' for annual
# averages. 
# For all area codes and titles see:
# http://www.bls.gov/cew/doc/titles/area/area_titles.htm
#
def qcewGetAreaData(year,qtr,area):
    urlPath = "http://data.bls.gov/cew/data/api/[YEAR]/[QTR]/area/[AREA].csv"
    urlPath = urlPath.replace("[YEAR]",year)
    urlPath = urlPath.replace("[QTR]",qtr.lower())
    urlPath = urlPath.replace("[AREA]",area.upper())
    httpStream = urllib.request.urlopen(urlPath)
    csv = httpStream.read()
    httpStream.close()
    return qcewCreateDataRows(csv)

### Creating the Schema

In [22]:
def generate_yaml_schema(schema_fields, schema_path):
    # Manually define the schema based on the schema_fields parameter
    schema = {"fields": schema_fields}
    with open(schema_path, 'w') as f:
        yaml.dump(schema, f, default_flow_style=False)
    print(f"Schema written to {schema_path}")
    return schema

### Cleaning Dataframe

In [23]:
def clean_data(temp_df):
    # Convert to DataFrame
    temp_df = pd.DataFrame(temp_df[1:], columns=temp_df[0])
    
    # Clean '"' from column names and data
    temp_df.columns = temp_df.columns.str.replace('"', '')
    temp_df = temp_df.replace('"', '', regex=True)
    
    # Drop the last row
    temp_df = temp_df.drop(temp_df.index[-1])
    return temp_df

### Saves data, schema, and resource files for each iteration

In [24]:
def exportDataResourceComponents(data, schema_fields, dataPath, schemaPath, resourcePath, year, region,qtr):
    print("Writing data to {}".format(dataPath))
    data.to_csv(dataPath, index=False)
    
    # Generating schema and capturing the schema dict
    schema_dict = generate_yaml_schema(schema_fields, schemaPath)
    
    acsResource = {
      "profile": "tabular-data-resource",
      "name": os.path.basename(dataPath).replace(".csv","").lower(),
      "path": os.path.basename(dataPath),
      "title": f"Quarterly Census of Employment and Wages data {year}_{region}_{qtr}",
      "description": f"Quarterly Census of Employment and Wages data for {region} in {qtr} of {year}.",
      "format": "csv",
      "mediatype": "text/csv",
      "encoding": "utf-8",
      "bytes": os.path.getsize(dataPath),
      "hash": f"md5:{compute_md5(dataPath)}",
      "rows": data.shape[0],
      "columns": data.shape[1],    
      "schema": schema_dict,
      "sources": [{
          "title": "Quarterly Census of Employment and Wages, U.S. Bureau of Labor Statistics",
          "path": QCEW_TABLE_DOC_URL
      }]
    }

    # Creating and saving the resource descriptor
    resource = Resource(acsResource)
    resource.to_yaml(resourcePath)
    print(f"Resource file written to {resourcePath}")
    
    # Validate the resource
    report = validate(resourcePath)
    if report.valid:
        print("Resource is valid")
    else:
        print("ERROR: Resource is NOT valid. Errors follow:")
        print(report.flatten(["code", "message"]))
        raise RuntimeError("Resource validation failed")

### Iterating through each year, region, and qtr combination

In [26]:
def iterate_years_and_regions(years, regions, qtrs, schema_fields):
    for year in years:
        for region in regions:
            for qtr in qtrs:
                TEMP_TABLE_NAME = f"morpc-qcew_{year}_{qtr}_{region}.csv"
                TEMP_TABLE_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_NAME)
                TEMP_TABLE_SCHEMA = TEMP_TABLE_NAME.replace(".csv",".schema.yaml")
                TEMP_TABLE_SCHEMA_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_SCHEMA)
                TEMP_TABLE_RESOURCE_NAME = TEMP_TABLE_NAME.replace(".csv",".resource.yaml")
                TEMP_TABLE_RESOURCE_PATH = os.path.join(OUTPUT_DIR, TEMP_TABLE_RESOURCE_NAME)

                # Getting data from API
                temp_data = qcewGetAreaData(str(year), qtr, region)

                # Cleaning data
                temp_df = clean_data(temp_data)

                # Calling function to save data, schema, and resource files
                exportDataResourceComponents(temp_df, schema_fields, TEMP_TABLE_PATH, TEMP_TABLE_SCHEMA_PATH, TEMP_TABLE_RESOURCE_PATH, year, region,qtr)
                
                print(f"DONE! Year: {year}, Region Code: {region}, Quarter: {qtr}, File: {TEMP_TABLE_PATH}")


# For annual QCEW data: ALL_QUARTERS["ANNUAL"], SCHEMA_PARAMETER["ANNUAL"] 
# For quarterly QCEW data: ALL_QUARTERS["QUATERS"], SCHEMA_PARAMETER["QUATERS"] 
iterate_years_and_regions(YEARS_PARAMETER["2020"], CONST_REGIONS_CODES["15-County Region"], ALL_QUARTERS["ANNUAL"], SCHEMA_PARAMETER["ANNUAL"])

Writing data to ./output_data\morpc-qcew_2020_a_39041.csv
Schema written to ./output_data\morpc-qcew_2020_a_39041.schema.yaml
Resource file written to ./output_data\morpc-qcew_2020_a_39041.resource.yaml
Resource is valid
DONE! Year: 2020, Region Code: 39041, Quarter: a, File: ./output_data\morpc-qcew_2020_a_39041.csv
Writing data to ./output_data\morpc-qcew_2020_a_39045.csv
Schema written to ./output_data\morpc-qcew_2020_a_39045.schema.yaml
Resource file written to ./output_data\morpc-qcew_2020_a_39045.resource.yaml
Resource is valid
DONE! Year: 2020, Region Code: 39045, Quarter: a, File: ./output_data\morpc-qcew_2020_a_39045.csv
Writing data to ./output_data\morpc-qcew_2020_a_39047.csv
Schema written to ./output_data\morpc-qcew_2020_a_39047.schema.yaml
Resource file written to ./output_data\morpc-qcew_2020_a_39047.resource.yaml
Resource is valid
DONE! Year: 2020, Region Code: 39047, Quarter: a, File: ./output_data\morpc-qcew_2020_a_39047.csv
Writing data to ./output_data\morpc-qcew_20